<a href="https://colab.research.google.com/github/darwinyusef/UsaHousingLab/blob/master/Actividad1ClasificacionRamdomAndTree14297510.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
!wget -O housing_train.csv 'https://raw.githubusercontent.com/darwinyusef/UsaHousingLab/master/housing_train.csv'

In [ ]:
df = pd.read_csv('/content/housing_train.csv')
df

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df_numericas = df.select_dtypes(include=['number'])
# @title Columnas categóricas
df_categoricas = df.select_dtypes(include=['object'])
df_numericas.nunique().shape, df_categoricas.nunique().shape

In [ ]:
df.nunique()

In [ ]:
# @title Tratamiento de missing

df['LotFrontage'] = df.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
df['Alley'] = df['Alley'].fillna('NoAcceso')
bsmt_cols = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
df[bsmt_cols] = df[bsmt_cols].fillna('SinSótano')
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)
garage_cols = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
df[garage_cols] = df[garage_cols].fillna('SinGaraje')
garage_cols = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
df[garage_cols] = df[garage_cols].fillna('SinGaraje')
df['MasVnrType'] = df['MasVnrType'].fillna('Ninguno')
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
df['FireplaceQu'] = df['FireplaceQu'].fillna('SinChimenea')
df['Fence'] = df['Fence'].fillna('SinCerca')
df['PoolQC'] = df['PoolQC'].fillna('SinPiscina')
df['MiscFeature'] = df['MiscFeature'].fillna('Ninguna')
missing_electrical = df['Electrical'].isnull().sum()
moda_electrical = df['Electrical'].mode()[0]
df['Electrical'] = df['Electrical'].fillna(moda_electrical)

# @markdown Resultado = 0
print(df.isnull().sum().sum())

## Ejercicio clasificación

tanto para Árboles como para random forest, se crean los siguientes grupos: grupo1 SalePrice menor o igual a 100.000, grupo2 SalePrice entre 100.001 y 500.000 y grupo3 SalePrice mayor o igual a 500001.


- OverallQual → Calidad general de la casa (Factor más importante en el precio).
- GrLivArea → Área habitable sobre el suelo (Casas más grandes valen más).
- GarageCars → Número de autos que caben en el garaje (Más espacio, más valor).
- TotalBsmtSF → Tamaño total del sótano (Espacio extra muy valioso).
- FullBath → Número de baños completos ( Mayor comodidad, mayor precio).
- YearBuilt → Año de construcción (Casas más nuevas suelen valer más).
- LotArea → Tamaño del terreno (Más terreno, más valor).
- GarageArea → Tamaño del garaje (Espacio adicional es un plus).

In [ ]:
características = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
X = df[características]


# 3. Crear la variable de clasificación según SalePrice
def categorizar_precio(precio):
    if precio <= 100000:
        return 1
    elif precio <= 500000:
        return 2
    else:
        return 3


df['PrecioGrupo'] = df['SalePrice'].apply(categorizar_precio)
y = df['PrecioGrupo']
y.shape

In [ ]:
import category_encoders as ce
# ========================== 3. Codificación de Variables Categóricas ==========================
# (Iris no tiene variables categóricas, pero aquí se muestra cómo hacerlo)
encoder = ce.OrdinalEncoder()
X_encoded = encoder.fit_transform(X)
X_encoded.shape

In [ ]:
df['PrecioGrupo'].value_counts()

In [ ]:
# Dividir el conjunto de datos en entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X_encoded, y, test_size=0.3, random_state=42, stratify=y)


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Aplicar SMOTE para BALANCEO de clases

# @ markdown - al aplicar balanceo notamos que la graficas cambiaron radicalmente disminuyendo a 0.10 la presición del modelo por esta causa retiro el SMOTE
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_entrenamiento, y_entrenamiento)

pd.Series(y_train_bal).value_counts()

In [ ]:
# Estandarizar las características
escalador = StandardScaler()
X_entrenamiento = escalador.fit_transform(X_entrenamiento)
X_prueba = escalador.transform(X_prueba)

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 150],  # Número de árboles en el bosque
    'max_depth': [None, 10, 20],  # Profundidad máxima del árbol
    'criterion': ['gini', 'entropy']  # Función de impureza
}

modelo = RandomForestClassifier(random_state=42)
modelo.fit(X_entrenamiento, y_entrenamiento)
y_pred_arbol = modelo.predict(X_prueba)

grid_search = GridSearchCV(modelo, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_entrenamiento, y_entrenamiento)

print(X_train_bal.shape, X_entrenamiento.shape)
print("\n")
print(pd.DataFrame(X_train_bal, columns=características).head())  # ¿Siguen presentes las características?

# 9. Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", grid_search.best_params_)

# 10. Evaluar el mejor modelo en el conjunto de prueba
mejor_modelo = grid_search.best_estimator_

In [ ]:
X_prueba.shape

In [ ]:
# ========================== 8. Evaluación del Modelo ==========================

y_train_pred = mejor_modelo.predict(X_entrenamiento)
y_test_pred = mejor_modelo.predict(X_prueba)

# Calculo el accuract en train


train_acc_ramdom = accuracy_score(y_true=y_entrenamiento,y_pred=y_train_pred)
# Calculo el accuract en test
test_acc_ramdom  = accuracy_score(y_true=y_prueba,y_pred=y_test_pred)

print("Exactitud en datos de entrenamiento:", train_acc_ramdom)
print("Exactitud en datos de predicción:", test_acc_ramdom)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# @title Matriz de Confusión para Ramdom Forest

# Definir las etiquetas de las clases
labels = ['<= 100000', '<= 500000', 'Otros']

cmrf = confusion_matrix(y_prueba, y_test_pred)
# Crear un DataFrame para la matriz de confusión
df_cm = pd.DataFrame(cmrf, index=labels, columns=labels)

# Graficar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(df_cm, annot=True, fmt='d', cmap='Reds')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión para ramdom forest')
plt.show()

df_cm

In [ ]:
# @title Reporte de Clasificación
report = classification_report(y_prueba, y_test_pred, output_dict=True)
# Convertir el informe en un DataFrame
df_report = pd.DataFrame(report).transpose()
df_report

In [ ]:
feature_scores = pd.DataFrame(pd.Series(mejor_modelo.feature_importances_, index=características).sort_values(ascending=False)).T
plt.figure(figsize=(12,6))
sns.barplot(data=feature_scores)

for index, value in enumerate(feature_scores.values.flatten()):
    plt.annotate(f'{value:.2f}', xy=(index, value), ha='center', va='bottom')


plt.title("Factores clave en la predicción de ramdom forest para este modelo")
plt.show()

In [ ]:
from matplotlib import pyplot as plt
df_report['f1-score'].plot(kind='line', figsize=(8, 4), title='f1-score')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# Seleccionar las métricas a graficar
metrics = ['precision', 'recall', 'f1-score']
palettes = ["magma", "flare", "viridis"]

# Graficar el informe de clasificación
df_report[metrics].iloc[:-1].plot(kind='bar', figsize=(10, 6))
plt.title('Informe de Clasificación')
plt.xlabel('Clases')
plt.ylabel('Puntuación')
plt.ylim(0, 1)
plt.legend(loc='lower right')
plt.grid(axis='y')
plt.show()

In [ ]:
sns.set(style="whitegrid")
for i, metric in enumerate(metrics):
    print(i)
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x=df_report.index[:-3], y=df_report[metric].iloc[:-3], palette=palettes[i])
    plt.title(f'{metric.capitalize()} por Clase')
    plt.xlabel('Clases')
    plt.ylabel(f'{metric.capitalize()}')
    plt.ylim(0, 1)
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                    textcoords='offset points')
    plt.show()

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

single_tree = modelo.estimators_[80]
num_trees = modelo.n_estimators
print(f"The number of trees in the Random Forest is: {num_trees}")
plt.figure(figsize=(20, 10))
tree.plot_tree(single_tree,
               filled=True,
               feature_names=X.columns,
               class_names=True)
plt.show()

In [ ]:
arbol = DecisionTreeClassifier(random_state=42)
arbol.fit(X_entrenamiento, y_entrenamiento)

y_pred_arbol = modelo.predict(X_entrenamiento)
y_test_arbol = modelo.predict(X_prueba)

#param_grid = {
#    'criterion': ['gini', 'entropy'],
#    'max_depth': [None, 10, 20, 30, 40, 50],
#    'min_samples_split': [2, 5, 10],
#    'min_samples_leaf': [1, 2, 4]
#}

# parametros del decision tree
param_grid = {'criterion': ['gini', 'entropy'], 'max_depth': [2, 3, 4, 5]}

grid_search = GridSearchCV(arbol, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_entrenamiento, y_entrenamiento)

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


# 9. Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", grid_search.best_params_)

# 10. Evaluar el mejor modelo en el conjunto de prueba
mejor_modelo_arbol = grid_search.best_estimator_


In [ ]:
y_train_pred_tree = mejor_modelo_arbol.predict(X_entrenamiento)
y_test_pred_tree = mejor_modelo_arbol.predict(X_prueba)


# Calculo el accuract en train
train_acc_arbol = accuracy_score(y_true=y_entrenamiento,y_pred=y_train_pred_tree)
# Calculo el accuract en test
test_acc_arbol  = accuracy_score(y_true=y_prueba,y_pred=y_test_pred_tree)

print("El accuracy en train es:",train_acc_arbol)
print("El accuracy en test es:",test_acc_arbol)

print(classification_report(y_prueba,y_test_pred_tree))

In [ ]:
# @title Matriz de Confusión para Ramdom Forest

# Definir las etiquetas de las clases
labels = ['<= 100000', '<= 500000', 'Otros']

cmrf=confusion_matrix(y_prueba,y_test_pred_tree)
print(cm)
accuracy_score(y_prueba,y_test_pred_tree)
# Crear un DataFrame para la matriz de confusión
df_cm = pd.DataFrame(cmrf, index=labels, columns=labels)

# Graficar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(df_cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión para ramdom forest')
plt.show()

df_cm

In [ ]:
feature_scores = pd.DataFrame(pd.Series(mejor_modelo_arbol.feature_importances_, index=características).sort_values(ascending=False)).T
plt.figure(figsize=(12,6))
sns.barplot(data=feature_scores)

for index, value in enumerate(feature_scores.values.flatten()):
    plt.annotate(f'{value:.2f}', xy=(index, value), ha='center', va='bottom')


plt.title("Factores clave en la predicción de la desition tree para este modelo")
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
tree.plot_tree(mejor_modelo_arbol,  # Use the DecisionTreeClassifier object directly
               filled=True,
               feature_names=características,  # Use the correct feature names from your dataset
               class_names=['1', '2', '3'])  # Assuming your classes are 1, 2, and 3
plt.show()

In [ ]:

print("Porcentaje en datos de entrenamiento en ramdom ", train_acc_ramdom)
print("El accuracy en train en tree",train_acc_arbol)

print("Porcentaje en datos de predicción en ramdom ", test_acc_ramdom)
print("El accuracy en test es tree",test_acc_arbol)

In [ ]:
valor_a = train_acc_ramdom
valor_b = train_acc_arbol
etiquetas = ['Ramdom Forest A', 'Desition Tree B']
valores = [valor_a, valor_b]

In [ ]:
# Crear el gráfico de pastel
colores = ['#ff9999','#66b3ff','#99ff99']  # Lista de colores en formato hexadecimal
fig, ax = plt.subplots()
wedges, texts, autotexts = ax.pie(valores, labels=etiquetas, autopct='%1.1f%%', startangle=90, colors=colores, wedgeprops=dict(width=0.3))
ax.axis('equal')
plt.title('Comparación de Efectividad de entrenamiento entre Ramdom Forest A y Desition Tree B')
plt.show()


In [ ]:
valor_a = test_acc_ramdom
valor_b = test_acc_arbol
etiquetas = ['Ramdom Forest A', 'Desition Tree B']
valores = [valor_a, valor_b]

In [ ]:
# Crear el gráfico de pastel
fig, ax = plt.subplots()
wedges, texts, autotexts = ax.pie(valores, labels=etiquetas, autopct='%1.1f%%', startangle=90, wedgeprops=dict(width=0.3))
ax.axis('equal')
plt.title('Comparación de Efectividad de test entre Ramdom Forest A y Desition Tree B')
plt.show()